#Initilisation du DataFrame

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
link = "https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv"
df_pk = pd.read_csv(link)

In [ ]:
df_pk.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [ ]:
df_pk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


#Data processing

In [ ]:
#On drop la 1ère colonne #
del df_pk['#']

In [ ]:
#Fonction get_dummies sur la colonne 'Type 1' 
df_pk=pd.concat([df_pk , pd.get_dummies( df_pk['Type 1'], dtype=int)], axis = 1)

In [ ]:
#Pour la colonne Type 2, 50% de valeurs NaN, donc pas utilisable: on supprime
del df_pk['Type 2']

In [ ]:
#Création d'une copie
df_pokemon=df_pk.copy()

In [ ]:
#On supprime tous les pokemons legendaires, car on en veut pas dans les recommandations
df_pokemon = df_pokemon[ df_pokemon['Legendary'] == False]

In [ ]:
df_pokemon.head()

,Name,Type 1,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,Bulbasaur,Grass,45,49,49,65,65,45,1,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Ivysaur,Grass,60,62,63,80,80,60,1,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Venusaur,Grass,80,82,83,100,100,80,1,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,Mega Venusaur,Grass,80,100,123,122,120,80,1,False,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,Charmander,Fire,39,52,43,60,50,65,1,False,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


#Creation et entrainement du model

In [ ]:
X =df_pokemon[['HP','Attack', 'Defense', 'Sp. Atk', 'Sp. Def','Speed', 'Generation','Bug', 'Dark','Dragon','Electric', 'Fairy', 'Fighting', 'Fire', 'Flying','Ghost','Grass','Ground', 'Ice','Normal','Poison','Psychic','Rock','Steel','Water']]
y=df_pokemon['Legendary']

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
  
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

Création d'un DataFrame contenant que les 6 pokemons du champion

In [ ]:
#Drop NaN dans la colonne 'Name'
df_pk.drop(df_pk[df_pk['Name'].isnull()].index, inplace=True)

In [ ]:
#On veut un DataFrame contenant que les lignes des pokemons de notre champion
df_pokemon2=df_pk[df_pk['Name'].str.contains("Mewtwo|Lugia|Rayquaza|Giratina|Dialga|Palkia")]

In [ ]:
#On drop les lignes en trop après notre 1er fltre
df_pokemon2.drop(df_pokemon2[df_pokemon2['Name']=='Mega Mewtwo X'].index, inplace=True)
df_pokemon2.drop(df_pokemon2[df_pokemon2['Name']=='Mega Mewtwo Y'].index, inplace=True)
df_pokemon2.drop(df_pokemon2[df_pokemon2['Name']=='Mega Rayquaza'].index, inplace=True)
df_pokemon2.drop(df_pokemon2[df_pokemon2['Name']=='Giratina Altered Forme'].index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Utilisation de la fonction kneighbors() pour trouver le ou les Pokémons le(s) plus proche(s) de chacun des Pokémon légendaires que le champion a l'habitude d'utiliser

In [ ]:
from sklearn.neighbors import NearestNeighbors

X2 = df_pokemon2[['HP','Attack', 'Defense', 'Sp. Atk', 'Sp. Def','Speed', 'Generation','Bug', 'Dark','Dragon','Electric', 'Fairy', 'Fighting', 'Fire', 'Flying','Ghost','Grass','Ground', 'Ice','Normal','Poison','Psychic','Rock','Steel','Water']]

result=model.kneighbors(X2)

In [ ]:
pkmPredit=[]
for i in range(6):
  pkmPredit.append(df_pokemon['Name'][df_pokemon.index==int(result[1][i])].values[0])

In [ ]:
df_pokemon2['Pokemon_de_remplacement']=pkmPredit

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_pokemon2[['Name','Pokemon_de_remplacement']]

,Name,Pokemon_de_remplacement
162,Mewtwo,Octillery
269,Lugia,Mantyke
425,Rayquaza,Mega Tyranitar
540,Dialga,Mega Blastoise
541,Palkia,Mega Charizard Y
545,Giratina Origin Forme,Abomasnow
